In [14]:
import json
import math

import ipyvuetify as v
from traitlets import (
    Unicode, observe, directional_link, 
    List, Int, Bool, CFloat, link, Any, CInt
)
from ipyleaflet import GeoJSON, TileLayer
from haversine import haversine

from shapely_geojson import dumps
import json
import datetime
import pandas as pd
import calendar

In [15]:
from sepal_ui.mapping import mapping as m

In [16]:
from component.scripts.scripts import *
from component.widget.custom_widgets import *
from component.frontend.styles import *
from component.message import cm

In [17]:
import os
from pathlib import Path
import geopandas as gpd
from traitlets import Unicode, link, observe
import ipyvuetify as v 
import sepal_ui.sepalwidgets as sw
from component.widget.custom_widgets import *

class AOI(v.Layout):
    
    shapefile = Unicode('').tag(sync=True)
    
    def __init__(self, statebar=None, **kwargs):
    
        super().__init__(**kwargs)
        
        self.gdf = None
        
        # Parameters
        self.out_path = Path('')
        self.json_path = None
        
        # Widgets
        self.shape_input = sw.FileInput(['.shp'], os.getcwd())
        
        self.w_state_bar = StateBar(done=True) if not statebar else statebar
        
        # Link behaviours 
        
        link((self.shape_input, 'file'), (self, 'shapefile'))
        
        # View
        
        self.children=[
            v.Layout(row=True, children=[
                self.shape_input,
            ])
        ]
        
        # Add a statebar if there is not provided an external one
        if not statebar: self.children = self.children + [self.w_state_bar]

    @observe('shapefile')
    def shape_to_geojson(self, change):
        """ Converts shapefile into Json file"""
        
        shp_file_path = Path(self.shapefile)
        if shp_file_path.suffix == '.shp':

            self.gdf = gpd.read_file(str(shp_file_path))
            self.gdf = self.gdf.to_crs("EPSG:4326")
            
            self.json_path = shp_file_path.parent/f'{shp_file_path.stem}.geojson'
            
            if not self.json_path.exists():
                self.w_state_bar.add_msg('Converting shape to GeoJSON', done=False)
                self.gdf.to_file(str(self.json_path), driver='GeoJSON')
                self.w_state_bar.add_msg('Done', done=True)
            else:
                self.w_state_bar.add_msg('Geojson file already created', done=True)
                
    def get_ipyleaflet_geojson(self):
        """Returns GeoJSON ipyleaflet object from Json file"""
        
        if self.json_path:
            self.w_state_bar.add_msg('Converting shape to GeoJSON', done=False)
            with open(self.json_path) as f:
                data = json.load(f)        
                ipygeojson = GeoJSON(
                    data=data,
                    name=self.json_path.stem, 
                    style={'color': 'green', 'fillOpacity': 0, 'weight': 3})

            self.w_state_bar.add_msg('Done', done=True)
        
            return ipygeojson
        else:
            self.w_state_bar.add_msg('There is not a shapefile selected.', done=True)


In [19]:
class PlanetTile(v.Flex):
    
    def __init__(self, planet_key, w_state=None, *args, **kwargs):
        
        self.class_='align-center mb-2'
        self.row = True
        
        super().__init__(*args, **kwargs)
        
        self.api_key = ''
        self.valid_planet = False
        self.client = None
        self.planet_key = planet_key
        
#         self.w_state = StateBar(done=True) if not w_state else w_state
        
        self.not_connected = Tooltip(
            widget=v.Icon(children=['mdi-circle'], color='red', x_small=True),
            tooltip='Not connected to Planet')
        
        self.connected = Tooltip(
            widget=v.Icon(children=['mdi-circle'], color='green', x_small=True),
            tooltip='Connected to Planet')
        
        self.w_api_key = PasswordField(v_model=self.api_key)
        w_api_btn = sw.Btn('Validate ', small=True,)
        
        w_api_key = v.Flex(class_='d-flex align-center mb-2', 
               row=True, 
               children =[self.w_api_key, w_api_btn]
        )
        
        self.children = [
            self.not_connected,
            w_api_key,
        ]
        # Events
        w_api_btn.on_event('click', self._validate_api_event)

        # Add a statebar if there is not provided an external one
            
    def _validate_api_event(self, widget, change, data):
        
        self.api_key = self.w_api_key.v_model
        
        planet_key = self.planet_key(self.api_key)
        self.client = planet_key.client()
        
        self.valid_planet = planet_key.is_active()
        
        if self.valid_planet:
            self.children.pop(0)
            self.children = [self.connected] + self.children
        else:
            self.children.pop(0)
            self.children = [self.not_connected] + self.children


In [20]:
planet_tile = PlanetTile(PlanetKey)

In [21]:
planet_tile

PlanetTile(children=[Tooltip(bottom=True, children=['Not connected to Planet'], v_slots=[{'name': 'activator',…

In [22]:
class Parameters(v.Card):
    
    sources = List().tag(sync=True)
    years = List(['2017', '2020']).tag(sync=True)
    month = CInt().tag(sync=True)
    
    def __init__(self, *args, **kwargs):
    
        self.class_ = 'pa-2'
        
        super().__init__(*args, **kwargs)
    
        self.w_state = StateBar(done=True)
        
        w_sources = v.Autocomplete(
            items=['Planet', 'Landsat', 'Sentinel'], 
            label='Imagery source',
            v_model = ['Planet'],
            chips=True, 
            clearable=True, 
            deletable_chips=True, 
            multiple=True
        )
        
        w_years = v.RangeSlider(
            tick_labels=list(str(x) for x in range(2010, 2021+1, 1)),
            v_model = ['2017', '2018'],
            min="2010",
            max="2021",
            ticks="always",
            tick_size="1"
        )
        
        w_month = v.Slider(ticks="always", tick_size="1", min=1, max=12, step=1,
                 track_color='primary', color='primary', thumb_color='grey', v_model=6,
                 tick_labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                              'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dec'])
        
        self.w_aoi = AOI(self.w_state)
        self.w_planet = PlanetTile(PlanetKey, w_state=self.w_state)
        
        self.children = [
            self.w_planet,
            self.w_aoi,
            w_sources,
            w_years,
            w_month,
            self.w_state,
        ]
        
        link((self, 'sources'),(w_sources, 'v_model'))
        link((self, 'years'),(w_years, 'v_model'))
        link((self, 'month'),(w_month, 'v_model'))
        

In [23]:
class MultiMap(v.Card):
    
    center = List([4,-74])
    zoom = CFloat(5)
    
    def __init__(self, parameters, **kwargs):
        
        super().__init__(**kwargs)
        
        self.param = parameters
        self.maps = {}

        self.map_count = 0
        self.w_state_bar = StateBar(done=True)
        self.w_nav_feats = DynamicSelect(label='Features')
        self.btn_get_maps = v.Btn(children=['Get Maps'],)
        self.btn_reload = v.Btn(children=['Reload imagery'],)
        
        # Events
        self.btn_get_maps.on_event('click', self.get_maps_widget)
        self.btn_reload.on_event('click', self.reload_event)
        
        # View
        
        self.children=[
            self.w_nav_feats,
            self.w_state_bar,
            self.btn_get_maps,
            self.btn_reload,
        ]
        
        # Events
        self.w_nav_feats.observe(self.zoom_to_feature, 'v_model')
        
    def _get_items(self):
        
        # Get json geometry from selected feature
        geom = json.loads(dumps(self.param.w_aoi.gdf.iloc[self.w_nav_feats.v_model].geometry.buffer(0.001, cap_style=3)))
        
        items = []
        # Get items for each year-month
        for year in range(self.param.years[0], self.param.years[1]+1):
            
            ini_date = datetime.datetime(year, self.param.month, 1)
            end_date = datetime.datetime(year, self.param.month, calendar.monthrange(year,self.param.month)[1])
            
            req = build_request(geom, ini_date, end_date, cloud_cover=.15)
            items.append(get_items('Planet', req, self.param.w_planet.client))
        
        return items
    
    def _prioritize_items(self):
                
        dates = self._get_items()
        print(len(dates))
        
        items_date = []
        for items in dates:
            it = [(item['properties']['item_type'], 
                      item['id'],
                      pd.to_datetime(item['properties']['acquired']).strftime('%Y-%m-%d-%H:%M')
                     ) for item in items[1]]
        
            items_df = pd.DataFrame(data=it, columns=['item_type', 'id', 'date'])
            items_df.sort_values(by=['item_type'])
            items_df.drop_duplicates(subset=['date', 'id'])
        
            # If more than one day is selected, get one image per day.

            items_df.date = pd.to_datetime(items_df.date)
            items_df = items_df.groupby(
                [items_df.date.dt.year, items_df.date.dt.day]
            ).nth(1).reset_index(drop=True)

            items_df = items_df.head(1)

            items_date.append(items_df)
        
        return items_date

    def add_layers(self):
        """Search planet imagery and add them to every map_"""
        
        # Validate whether Planet API Key is valid,
        # and if there is already selected coordinates.
        
        items_df = self._prioritize_items()
        print(items_df)
        
        for map_, items_df in zip(self.maps.values(), items_df):

            # remove all previous loaded assets
            remove_layers_if(map_, 'attribution', 'Planet')

            for i, row in items_df.iterrows():
                layer = TileLayer(
                    url=f'https://tiles0.planet.com/data/v1/{row.item_type}/{row.id}/{{z}}/{{x}}/{{y}}.png?api_key={self.param.w_planet.api_key}',
                    name=f'{row.item_type}, {row.date}',
                    attribution='Planet'
                )
                layer.__setattr__('_metadata', {'type':row.item_type, 'id':row.id})
                if row.id not in [layer._metadata['id'] for layer in map_.layers if hasattr(layer, '_metadata')]:
                    map_+layer
    
    def reload_event(self, widget, event, data):
        
        self.add_layers()
    
    def zoom_to_feature(self, change, zoom_out=3):

        """Get coordinates for the current feature and zoom it to maps"""

        geom = self.param.w_aoi.gdf.loc[change['new']].geometry

        min_lon, min_lat, max_lon, max_lat = geom.bounds
        lon, lat = geom.centroid.x, geom.centroid.y

        tl = (min_lon, max_lat)
        bl = (min_lon, min_lat)
        tr = (max_lon, max_lat)
        br = (max_lon, min_lat)

        maxsize = max(haversine(tl, br), haversine(bl, tr))
        lg = 40075 # number of displayed km at zoom 1
        zoom = 1
        while lg > maxsize:
            zoom += 1
            lg /= 2

        if zoom_out > zoom:
            zoom_out = zoom - 1

        self.zoom = zoom-zoom_out
        self.center = (lat, lon)
        
        self.add_layers()
                
    def _get_loading_maps_layout(self, max_cols, complete_rows, tails):
        """Get loading layout, while maps are ready to display"""
        
        w_loading_maps = v.Content(
            _metadata={'type':'loading_map_tiles'},
            children= 
                [v.Row(
                    children=[
                        v.Col(
                            children=[v.SkeletonLoader(type="image")]
                        )for col in range(max_cols)]
                ) for row in range(complete_rows)] + \
                [v.Row(
                    children=[
                        v.Col(
                            children=[v.SkeletonLoader(type="image")]
                        ) for tail in range(tails)
                    ]
                )]
        )
        
        return w_loading_maps
    
    def _add_features_to_map(self):
        
        features = self.param.w_aoi.get_ipyleaflet_geojson()
        self.w_nav_feats.items = self.param.w_aoi.gdf.index.to_list()
        
        for map_ in self.maps.values():
            map_+features
        
    def get_maps_layout(self):
        """Instantiate SepalMaps in a grid"""
        
        # Get the number of maps to be displayed
        start, end = self.param.years
        max_cols = 4
        n_maps = len(range(start,end+1,1))
        complete_rows = math.floor(n_maps/max_cols)
        tails = n_maps%max_cols
        
        # Get childrens with metadata
        childs = [(i, ch._metadata['type']) for i, ch in enumerate(self.children) if ch._metadata]
        tiles = []
        if childs:
            idxs, tiles = zip(*childs)
        
        if 'map_tiles' not in tiles:
            # If not maps in view, display loading_tiles
            self.children = self.children + [self._get_loading_maps_layout(max_cols, complete_rows, tails)]
            
        else:
            self.children = [chd for idx, chd in enumerate(self.children) if idx not in idxs] + \
            [self._get_loading_maps_layout(max_cols, complete_rows, tails)]
        
        self.w_state_bar.add_msg('Loading maps...', done=False)
        
        self.w_maps = v.Content(
            _metadata={'type' : 'map_tiles'},
            children=[
                v.Row(
                    children=[
                        v.Col(
                            children=[self.create_map()]
                    ) for col in range(max_cols)]
            ) for row in range(complete_rows)] + \
            [v.Row(
                 children=[
                     v.Col(
                         children=[self.create_map()]
                    ) for tail in range(tails)
                ]
            )]
        )
        
        self._add_features_to_map()
        
        
        # Replace loading squares with loaded maps
        self.children = self.children[:-1] + [self.w_maps]
    
        self.w_state_bar.add_msg('Done', done=True)
        
    def get_maps_widget(self, widget, event, data):
        
        self.get_maps_layout() 

    
    def create_map(self):
        
        self.map_count+=1
        
        self.maps[self.map_count] = m.SepalMap()
        # TODO: Change map height
        
        self.maps[self.map_count].layout.height = '400px'
        
        link((self, 'center'), (self.maps[self.map_count], 'center'))
        link((self, 'zoom'), (self.maps[self.map_count], 'zoom'))
        
        return self.maps[self.map_count]

In [24]:
parameters = Parameters()

In [25]:
parameters

Parameters(children=[PlanetTile(children=[Tooltip(bottom=True, children=['Not connected to Planet'], v_slots=[…

In [26]:
mp = MultiMap(parameters=parameters)
mp

MultiMap(children=[DynamicSelect(children=[Btn(children=[Icon(children=['mdi-chevron-left'], left=True), 'prev…

In [ ]:
# Todo: Create messages
# Every map is pointing to a previous one
# All polygon must be contained with image footprint
# Create reload button
# toggle geojson with one button
# Set year in top of map
# Use multithread to search images
# Validate ok message